# Part 1.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

data_path = '/home/jovyan/STA365/tsla_2014_2023.csv'
df = pd.read_csv(data_path)

df['price_increase_next_day'] = (df['next_day_close'] > df['close']).astype(int)
df.drop(columns='next_day_close', inplace=True) 

X = df.drop(columns=['date', 'price_increase_next_day'])
y = df['price_increase_next_day']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n, p = X_scaled.shape

with pm.Model() as logistic_model:
    betas = pm.Normal('betas', mu=0, sigma=10, shape=p) 
    intercept = pm.Normal('intercept', mu=0, sigma=10) 

    linear_component = pm.math.dot(X_scaled, betas) + intercept
    p_outcome = pm.math.sigmoid(linear_component)  
    y_obs = pm.Bernoulli('y_obs', p=p_outcome, observed=y.values)
    
    trace = pm.sample(1000, target_accept=0.95, return_inferencedata=False) 

# Part 2.
In Bayesian statistics, we express our beliefs about parameters using prior distributions. Incorporating these priors into our model, the posterior distribution reflects a regularization analogous to Lasso and Ridge.

For a parameter vector $\beta$:

A Normal prior corresponds to L2 regularization (Ridge): $\beta_j \sim \mathcal{N}(0, \tau^2)$ where $\tau^2 = 1/\lambda$. The log posterior distribution (omitting constant terms for simplicity) with $\sigma=1$ is proportional to: $-\frac{1}{2}(y - X\beta)^T(y - X\beta) - \lambda ||\beta||_2^2$. This resembles the Ridge regression loss function, where the penalty term is the squared L2 norm of $\beta$.

A Laplace prior corresponds to L1 regularization (Lasso):$\beta_j \sim \text{Laplace}(0, b)$ with $b = 1/\sqrt{\lambda}$. The log posterior, again omitting constants and with $\sigma=1$, is proportional to: $-\frac{1}{2}(y - X\beta)^T(y - X\beta) - \lambda ||\beta||_1$. This mirrors the Lasso regression loss function, where the penalty term is the L1 norm of $\beta$.

Bayesian inference updates parameter beliefs by combining prior information with data, yielding a posterior distribution. Unlike optimizing a single estimate, Bayesians sample from this distribution to understand parameter uncertainty. The prior effectively regularizes the estimation, guiding it towards plausible values, akin to how frequentist regularization penalizes model complexity.


# Part 3

In [ ]:
import pymc as pm
import numpy as np


with pm.Model() as robust_model:
    beta = pm.Normal('beta', mu=0, sd=10, shape=X.shape[1])
    intercept = pm.Normal('intercept', mu=0, sd=10)
    w = pm.HalfNormal('w', sd=10)

    nu = pm.Exponential('nu', 1/29) + 1

    y_pred = pm.StudentT('y_pred', mu=intercept + pm.math.dot(X, beta),
                         sd=w, nu=nu, observed=y)
    

    trace = pm.sample(1000, return_inferencedata=True)

with robust_model:
    ppc = pm.sample_posterior_predictive(trace, var_names=['y_pred'])


residuals = y.values - ppc['y_pred'].mean(axis=0)
std_residuals = residuals / residuals.std()

outliers = np.where(np.abs(std_residuals) > 2)[0]  
